In [ ]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy
!pip install einops # Falcon dependency

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-jfht7cax
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jfht7cax
  Resolved https://github.com/huggingface/transformers.git to commit f92cc7034a49959b247a46a210b912e56a6f977d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-tdflwk46
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-tdflwk46


In [6]:
!nvidia-smi
!free

Mon Aug 21 13:59:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  Off  | 00000000:00:07.0 Off |                    0 |
| N/A   33C    P0    36W / 250W |  31422MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100S-PCI...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   

In [ ]:
# Empty model cache
!pip install huggingface_hub["cli"]
!huggingface-cli delete-cache

In [1]:
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [2]:
model_name = 'Salesforce/xgen-7b-8k-inst'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True
)

A new version of the following files was downloaded from https://huggingface.co/Salesforce/xgen-7b-8k-inst:
- tokenization_xgen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [21]:
MAX_CHARACTER_CHUNK_SIZE = 2000 * 3

def chunkize(text):
    
    chunks = [] # Final list of transcript chunks. This makes up the loop invariant
    utterances = text.split('\n') # Transcript is split into sentences
    utterances.reverse() # Reverse everything!!
    current_chunk = ''

    # While there is still an utterance to process
    while len(utterances) > 0:
        utterance = utterances.pop()

        # Add to current chunk and proceed to next
        if len(current_chunk) + len(utterance) <= MAX_CHARACTER_CHUNK_SIZE:
            if len(current_chunk) > 0:
                current_chunk += '\n'
            current_chunk += utterance
        
        # Utterance is larger than maximum chunk size
        elif len(current_chunk) == 0:
            chunks.append(current_chunk)
            chunks.append(utterance)
            current_chunk = ''

        # Current chunk is big enough, append to list and create new one
        else:
            chunks.append(current_chunk)
            current_chunk = utterance

    if len(current_chunk) > 0:
        chunks.append(current_chunk)

    return chunks

In [22]:
# Load sample
sample_file = open('input/texts/sample_11_10498.txt', 'r', encoding='utf-8')
sample = sample_file.read()
sample_file.close()

sample_chunks = chunkize(sample)

In [13]:
def output_from_sequence(output):

    # Remove preprompt
    delimiter = '### Assistant:'
    output = delimiter.join(output.split(delimiter)[1:])

    # Remove white spaces in front of summary
    while len(output) > 0 and output[0] == ' ':
        output = output[1:]

    # Remove <|endoftext|>
    eos_token = '\n<|endoftext|>'
    output = output[:-len(eos_token)]

    return output

In [36]:
PROMPT_TEMPLATE = """A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Continue the summary of the first part of a dialogue, based on the second part of the same dialogue.

Summary:
During a team meeting, the speaker discusses the agenda for an upcoming presentation on remote control concepts. They inform the team that the meeting will follow a similar structure to the previous one, where each member will present their progress. The goal is to finalize key remote control concepts, including component properties, materials, user interface, and market trends. The first presenter discusses the internal components of a remote control, including the circuit board, chip, and LED device. They emphasize the flexibility in circuit board configurations and propose various battery options, such as standard cells, dynamos, kinetic energy, and solar cells. The speaker also lists different case shapes and materials, like flat, curved, plastic, rubber, wood, and titanium. They note that titanium is limited by its applicability to curved surfaces. The meeting continues with discussions on potential interface options.

Second part:
{text}

### Assistant:"""

In [38]:
prompt = PROMPT_TEMPLATE.format(text=sample_chunks[1])
input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
input_length = len(input_ids['input_ids'][0])
print('Input Length:', input_length)

duration = time.time()
sample = model.generate(**input_ids, do_sample=True, max_new_tokens=500, top_k=100, eos_token_id=50256, temperature=0.3)
duration = time.time() - duration

print('Duration:', duration)
output = tokenizer.decode(sample[0]).strip()
output = output_from_sequence(output)
print(output)

Input Length: 1875
Duration: 11.162116527557373
The speaker continues to discuss the agenda for the upcoming presentation on remote control concepts. They mention that the meeting will follow a similar structure to the previous one, with each member presenting their progress. The goal is to finalize key remote control concepts, including component properties, materials, user interface, and market trends. The speaker lists various internal components of a remote control, including the circuit board, chip, and LED device. They also mention various case shapes and materials, such as flat, curved, plastic, rubber, wood, and titanium. They note that titanium is limited by its applicability to curved surfaces. The meeting continues with discussions on potential interface options, including push-buttons and scroll wheels. The speaker also mentions that the company has started making a sample sender and a sample speaker. They also mention that the company has developed a new user interface tha